In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ta
import vnstock as vn

In [8]:
# Define your parameters
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
OBV_PERIOD = 5
initial_investment = 120000000
backup_amount_initial = 40_000_000


companies_vn30 = [
    'SSI', 'BCM','VHM','VIC','VRE','BVH','POW','GAS','ACB','BID',
'CTG','HDB','MBB','SSB','SHB','STB','TCB','TPB','VCB','VIB','VPB','HPG',
'GVR','MSN','VNM','SAB','VJC','MWG','PLX','FPT']

win_rate = 0.7977011494
loss_rate = 1 - win_rate
mean_profit = 0.15
mean_loss = 0.1

In [9]:

def kelly_criterion(p, q, profit, loss):
    return (p/loss) - (q/profit)

In [10]:
kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)

6.62835249

In [4]:
def calculate_indicators(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['OBV'] = ta.volume.OnBalanceVolumeIndicator(df['close'], df['volume']).on_balance_volume()
    df['OBV_Slope'] = df['OBV'].diff(periods=OBV_PERIOD)
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)

    return df

def obv_strategy(df):
    if df.empty:
        return df

    df['Signal'] = 0

    # Buy signals
    df.loc[(df['Previous_RSI'] < RSI_OVERSOLD) & (df['RSI'] >= RSI_OVERSOLD) & (df['OBV_Slope'] > 0), 'Signal'] = 1


    # Sell Signals: 
    df.loc[(df['Previous_RSI'] > RSI_OVERBOUGHT) & (df['RSI'] <= RSI_OVERBOUGHT) & (df['OBV_Slope'] < 0), 'Signal'] = -1


    return df

In [5]:
def simulate_investment(ticker, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction):
    try:
        data = vn.stock_historical_data(ticker, '2021-01-01', '2024-01-01', resolution='1D', type='stock')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = obv_strategy(data)

        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        cash = initial_investment
        holdings = 0
        backup_amount = backup_amount_initial
        portfolio_values = []
        overspend = 0

        f_star = 0.25 * kelly_criterion(win_rate, loss_rate, mean_profit ,mean_loss)

        for i in range(len(data)):
            if data.index[i] in buy_signals:
                allocation = cash * f_star
                if allocation > 0:
                    shares_to_buy = int(allocation // data['close'][i])
                    if shares_to_buy > 0:
                        total_cost = shares_to_buy * data['close'][i]
                        if cash >= total_cost:
                            cash -= total_cost
                        else:
                            cash_needed = total_cost - cash
                            if cash_needed <= backup_amount:
                                cash = 0
                                backup_amount -= cash_needed
                                overspend += cash_needed
                            else: 
                                shares_to_buy = int((cash + backup_amount) // data['close'][i])
                                cash_needed = shares_to_buy * data['close'][i] - cash
                                cash = 0
                                backup_amount -= cash_needed
                                overspend += cash_needed
                        holdings += shares_to_buy
                        

            if data.index[i] in sell_signals and holdings > 0:
                current_price = data['close'].iloc[i]
                shares_to_sell = int(holdings * sell_fraction)
                if shares_to_sell > 0:
                    if shares_to_sell > holdings:
                        shares_to_sell = holdings
                    revenue = shares_to_sell * current_price 
                    cash += revenue
                    holdings -= shares_to_sell
                    # print(f"Selling {shares_to_sell} shares at {current_price} on {data.index[i]}")
                    #Check profit and repay backup amount if possible 
                    if cash > overspend:
                        repayment_amount = overspend
                        cash -= repayment_amount
                        overspend = 0
                        backup_amount += repayment_amount

            current_value = cash + holdings * data['close'].iloc[i] + backup_amount
            portfolio_values.append(current_value)
            # print(f"Day {data.index[i]}: Cash: {cash}, Holdings: {holdings}, Current Value: {current_value}")

        data['Portfolio_Value'] = portfolio_values
        data['Accumulated_Profit'] = data['Portfolio_Value'] - (initial_investment+backup_amount)

        return data
    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()

In [6]:
def backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction):
    results = []
    for company in companies_vn30:
        result = simulate_investment(company, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction)
        if not result.empty:
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / (initial_investment+backup_amount_initial)  * 100
            })
    return pd.DataFrame(results)

In [7]:
sell_fraction = 0.4 * kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
results_df = backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, sell_fraction)

# Save results to CSV
results_df.to_csv('result/OBV_kelly.csv', index=False)

# Print results
print(results_df)

average_rate_of_return = results_df['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)

C:\Users\Admin\AppData\Local\Temp\ipykernel_26852\4284494037.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_26852\2281030537.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shares_to_buy = int(allocation // data['close'][i])
C:\Users\Admin\AppData

   Company  Final Portfolio Value  Total Profit  Rate of Return
0      SSI              160694540        694540        0.434088
1      BCM              212586344      92539100       57.836937
2      VHM              156712230      36686400       22.929000
3      VIC              162986096      42968400       26.855250
4      VRE              169262950       9262950        5.789344
5      BVH              140291950      20264500       12.665313
6      POW              187228696      67222500       42.014063
7      GAS              233859050      73859050       46.161906
8      ACB              247327530     127325360       79.578350
9      BID              306119250     186100200      116.312625
10     CTG              171179320      51163600       31.977250
11     HDB              288324640     168320900      105.200563
12     MBB              237089740      77089740       48.181088
13     SSB              261056060     141035800       88.147375
14     SHB              179067000      5